# KeplerGL PersonTripOD Colaboratory

[Geo+Python ゆるハンズオン](https://udceasttokyo.connpass.com/event/147710/)に参加してきました<br>
Mamiさんの[KeplerGL_PersonTripOD.ipynb](https://github.com/mamix1116/geo_python_cookbook/blob/master/notebook/KeplerGL_PersonTripOD.ipynb)を、<br>
Colaboratoryで再現してみました(完全にはできてないけど〜汗)

### Thanks
- Mami Enomoto [@mamix1116](https://twitter.com/mamix1116) さん
- Kunihiko Miyoshi [@colspan](https://github.com/colspan) さん

## GeoPandas

In [0]:
!pip install geopandas
!apt-get install libproj-dev proj-data proj-bin
!apt-get install libgeos-dev
!pip install geoplot

## GoogleDrive

In [0]:
from google.colab import drive
from google.colab import files
drive.mount('/content/drive')

# 1. Go to this URL in a browser:の右にあるURLをクリック
# 2. アカウントの選択
# 3. 右下の「許可」をクリック
# 4. コードをコピー(アイコンがあるのでクリック)
# 5. Enter your authorization code:にペーストしてENTER

## import

In [0]:
%matplotlib inline

import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

## データ

### 交通流動量 パーソントリップＯＤ量データ をDLする
- 三大都市圏パーソントリップ調査
- https://www.tokyo-pt.jp/person/01
- パーソントリップ調査とは、「どのような人が」「どのような目的で」「どこからどこへ」「どのような交通手段で」移動したか

### 国土数値情報からDL

- 交通流動量 パーソントリップＯＤ量データ（東京圏）
- http://nlftp.mlit.go.jp/ksj/jpgis/datalist/KsjTmplt-S05-b.html
- S05-b-10_SYUTO.zipをダウンロード

## GoogleDriveに3つのファイルを保存

- S05-b-10_SYUTO-1-g_PersonTripODAmount.dbf
- S05-b-10_SYUTO-1-g_PersonTripODAmount.shp
- S05-b-10_SYUTO-1-g_PersonTripODAmount.shx

#### ※ .shpだけだとエラーにまります

In [0]:
# ファイルを読み込む
# パスは左のファイルから.shpファイルまでたどって、右クリック「パスをコピー」してペースト

df_personTripOD = gpd.read_file('/content/drive/xxxxxx/S05-b-10_SYUTO-1-g_PersonTripODAmount.shp')
df_personTripOD.head()

- 'S05b_002', 調査年度
- 'S05b_003', 発ゾーンコード
- 'S05b_004', 着ゾーンコード
- 'S05b_010', 鉄道-合計トリップ数（交通手段「鉄道」、目的合計のトリップ数　）
- 'S05b_035', 全トリップ数（全交通手段、目的合計のトリップ数）
- 'geometry'

In [0]:
# 必要なカラムを確認　
df_personTripOD[['S05b_002', 'S05b_003', 'S05b_004', 'S05b_010', 'S05b_011', 'S05b_012']]

In [0]:
len(df_personTripOD)

## 東京都市圏ゾーンコード

- 0010    東京都    千代田区    皇居外苑
- 0010    東京都    千代田区    千代田
- 0010    東京都    千代田区    北の丸公園
- 0010    東京都    千代田区    大手町１丁目
- 0010    東京都    千代田区    大手町２丁目
- 0010    東京都    千代田区    丸の内１丁目
- 0010    東京都    千代田区    丸の内２丁目
- 0010    東京都    千代田区    丸の内３丁目
- 0011    東京都    千代田区    永田町１丁目
- 0011    東京都    千代田区    永田町２丁目
- 0011    東京都    千代田区    隼町
- 0011    東京都    千代田区    平河町１丁目

http://nlftp.mlit.go.jp/ksj/jpgis/codelist/TokyoAreaZoneCd.html

In [0]:
# 0010が起点（発ゾーン）のODだけを抽出して確認する
df_OD_selected = df_personTripOD.query('S05b_003 == "0010"')

In [0]:
len(df_OD_selected)

In [0]:
# 地図描写
df_OD_selected.plot()

In [0]:
# 必要なカラムを抽出
df_personTripOD2 = df_personTripOD[['S05b_003', 'S05b_004',  'S05b_010', 'S05b_035', 'geometry']]
df_personTripOD2.columns = ['zone01', 'zone02', 'trip_train', 'trip_all', 'geometry']
df_personTripOD2.head()

In [0]:
# 起点と終点の緯度経度
df_personTripOD3 = df_personTripOD2.assign(
    lng_to=lambda df_personTripOD2: df_personTripOD2.apply(lambda row: row['geometry'].xy[0][0], axis=1) ,
    lng_from=lambda df_personTripOD2: df_personTripOD2.apply(lambda row: row['geometry'].xy[0][1], axis=1) ,
    lat_to=lambda df_personTripOD2: df_personTripOD2.apply(lambda row: row['geometry'].xy[1][0], axis=1) ,
    lat_from=lambda df_personTripOD2: df_personTripOD2.apply(lambda row: row['geometry'].xy[1][1], axis=1) 
)

In [0]:
df_personTripOD3.head()

## KeplerGL for Jupyter

- KeplerGLのJupyterウィジェット
- https://github.com/keplergl/kepler.gl/tree/master/bindings/kepler.gl-jupyter
- Kepler GL https://kepler.gl/

#### ※Colaboratoryではipywidgetsが使えない

### ユーザーガイド
https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md

### 必要環境
- Python >= 2
- ipywidgets >= 7.0.0

### JupyterLabの必要環境
- Node > 8.15.0
- Python 3
- JupyterLab>=1.0.0

In [0]:
!node --version

In [0]:
# !apt install node@8

In [0]:
!npm i -g n
!n v12.13.0
!node --version

In [0]:
!pip install keplergl

In [0]:
!pip install jupyterlab

In [0]:
!jupyter labextension install @jupyter-widgets/jupyterlab-manager keplergl-jupyter

In [0]:
from keplergl import KeplerGl

## GoogleDriveにkeplerGL設定ファイルを保存
- KeplerGL_PersonTripOD_config.txt
- 下記を保存
```
{'config': {'mapState': {'bearing': -22.111111111111118,
   'dragRotate': True,
   'isSplit': False,
   'latitude': 35.67180136580014,
   'longitude': 139.75256414941893,
   'pitch': 49.00348473774453,
   'zoom': 11.139992472965828},
  'mapStyle': {'mapStyles': {},
   'styleType': 'dark',
   'threeDBuildingColor': [9.665468314072013,
    17.18305478057247,
    31.1442867897876],
   'topLayerGroups': {},
   'visibleLayerGroups': {'3d building': False,
    'border': False,
    'building': True,
    'label': True,
    'land': True,
    'road': True,
    'water': True}},
  'visState': {'filters': [],
   'interactionConfig': {'brush': {'enabled': True, 'size': 1.1},
    'tooltip': {'enabled': False,
     'fieldsToShow': {'personTripOD': ['zone01',
       'zone02',
       'trip_train',
       'trip_all']}}},
   'layerBlending': 'normal',
   'layers': [{'config': {'color': [225, 49, 106],
      'columns': {'lat0': 'lat_from',
       'lat1': 'lat_to',
       'lng0': 'lng_from',
       'lng1': 'lng_to'},
      'dataId': 'personTripOD',
      'isVisible': True,
      'label': 'personTripOD',
      'textLabel': [{'alignment': 'center',
        'anchor': 'start',
        'color': [255, 255, 255],
        'field': None,
        'offset': [0, 0],
        'size': 18}],
      'visConfig': {'colorRange': {'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300'],
        'name': 'Global Warming',
        'type': 'sequential'},
       'opacity': 0.8,
       'sizeRange': [0, 1],
       'targetColor': [207, 216, 244],
       'thickness': 1}},
     'id': '4hj0mq',
     'type': 'arc',
     'visualChannels': {'colorField': None,
      'colorScale': 'quantile',
      'sizeField': {'name': 'trip_train', 'type': 'integer'},
      'sizeScale': 'linear'}}],
   'splitMaps': []}},
 'version': 'v1'}
 ```

In [0]:
# 設定ファイルを読み込む
# パスは左のファイルから.shpファイルまでたどって、右クリック「パスをコピー」してペースト

myconfig = open('/content/drive/xxxxxx/KeplerGL_PersonTripOD_config.txt').read()

In [0]:
import ast

In [0]:
map_personTripOD_curve = KeplerGl(height=600, config=ast.literal_eval(myconfig))
map_personTripOD_curve.add_data(data=df_personTripOD3, name='personTripOD')
map_personTripOD_curve

In [0]:
map_personTripOD_curve.config

In [0]:
# 設定ファイル
# config = map_personTripOD_curve.config
# with open("KeplerGL_PersonTripOD_config.txt", "w") as f:
# print(config, file=f)

In [0]:
# HTML に出力
map_personTripOD_curve.save_to_html(file_name="my_PersonTripOD_map.html")

from IPython.display import display_html
with open('my_PersonTripOD_map.html') as f:
    display_html(f.read(), raw=True)

## Ngrok

In [0]:
# Ngrok
! wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip -d /opt ngrok-stable-linux-amd64.zip 
! rm ngrok-stable-linux-amd64.zip 
! echo "web_addr: 4045" > /content/config.yml

In [0]:
# publish tileset
! pkill ngrok
get_ipython().system_raw('/opt/ngrok http -config=/content/config.yml 8888 &')
! sleep 1

In [0]:
import requests
import json
r = requests.get("http://localhost:4045/api/tunnels", headers={"content-type": "application/json"})
base_url = r.json()['tunnels'][0]['public_url'].replace('http://', 'https://')
base_url

##### 上記の、https://xxxxxxxx.ngrok.ioで描画

In [0]:
# start server
! npm -g install http-server
get_ipython().system_raw('http-server -p 8888 --cors &')
! sleep 1

## iframe

In [0]:
from IPython.core.display import display, HTML
display_html('<iframe src="' +base_url+'/my_PersonTripOD_map.html' + '" width="1000px" height="500px"></iframe>', raw=True)